<a href="https://colab.research.google.com/github/RicoStaedeli/NLP2025_CQG/blob/main/Development/Inference/ZeroShot/2_Baseline_CQS_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Baseline Predictions
In this file we generate the baseline predictions

## Setup

In [5]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [7]:
from unsloth import FastLanguageModel
import torch
from google.colab import userdata, drive
import logging

Error importing huggingface_hub.repocard: partially initialized module 'yaml' has no attribute 'error' (most likely due to a circular import)
Error importing huggingface_hub.repocard_data: partially initialized module 'yaml' has no attribute 'error' (most likely due to a circular import)
Error importing huggingface_hub.hf_api: partially initialized module 'yaml' has no attribute 'error' (most likely due to a circular import)
Error importing huggingface_hub.utils: partially initialized module 'yaml' has no attribute 'error' (most likely due to a circular import)
Error importing huggingface_hub.repocard: partially initialized module 'yaml' has no attribute 'error' (most likely due to a circular import)


ImportError: cannot import name 'DatasetCard' from 'huggingface_hub' (/usr/local/lib/python3.11/dist-packages/huggingface_hub/__init__.py)

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
token = userdata.get('GITHUB')
repo_url = f"https://{token}@github.com/RicoStaedeli/NLP2025_CQG.git"

!git clone {repo_url}

In [ ]:
os.chdir("NLP2025_CQG")
!ls

In [3]:
################################################################################
#######################   PATH VARIABLES        ################################
################################################################################

test_dataset_path = "Data/Processed/test.csv"

results_path = "Evaluation/Results/"
os.makedirs(results_path, exist_ok=True)

log_base_path = f"Logs/"
os.makedirs(log_base_path, exist_ok=True)

log_path = log_base_path + "2_baseline_generation.log"


################################################################################
#######################   STATIC VARIABLES      ################################
################################################################################

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

In [4]:
# Setup logger manually
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

# Create file handler (only if not already added)
if not logger.handlers:
    fh = logging.FileHandler(log_path)
    fh.setLevel(logging.INFO)
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    fh.setFormatter(formatter)
    logger.addHandler(fh)

# Detect device
device = torch.device(
    "mps" if torch.backends.mps.is_available()
    else "cuda" if torch.cuda.is_available()
    else "cpu"
)

# Log the device info
logger.info("--------  Start with Baseline Generation  -------------")
logger.info(f'Device selected: {device}')
logger.info(f'Results Path: {results_path}')
logger.info(f'Log Path: {log_path}')
logger.info("--------------------------------------------------------")

INFO:__main__:--------  Start with Baseline Generation  -------------
INFO:__main__:Device selected: cuda
INFO:__main__:Results Path: Evaluation/Results/
INFO:__main__:Log Path: Logs/2_baseline_generation.log
INFO:__main__:--------------------------------------------------------


## Zero Shot prompting
In this section we genererate critical questions with different pretrained vanilla models. We use this generated questions as a baseline to compare it against our results. The following models were used to generate the baseline results:
- LLama 3.1 8B Instruct
- Qwen 2.5 7B Instruct

In [6]:
models = [
    {
        "model_name": "Meta-Llama-3.1-8B-Instruct",
        "model_path": "unsloth/Meta-Llama-3.1-8B-Instruct",
        "output_file": results_path + f"results_Meta-Llama-3.1-8B-Instruct.csv",
    }
]

In [7]:
def load_model(model_name):
  model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
  )
  return model, tokenizer

In [8]:
input_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You generate critical questions that should be raised before accepting the arguments in this text.

### Input:
{}

### Response:
{}"""

In [9]:
model,tokenizer = load_model(models[0]['model_path'])

==((====))==  Unsloth 2025.5.4: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.5k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [10]:
FastLanguageModel.for_inference(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
    (layers): ModuleList(
      (0): LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((409

In [12]:
text = input_prompt.format(
            "Minimum suffering, or no suffering? I initially became vegan because I started off wanting to be an ethical omnivore, who only eats meat from animals that lived well and didn't suffer when they were killed. After watching some videos about slaughterhouses (please watch Dominion, it's very eye opening) I determined that only home kill meat could be reasonably close to no suffering. I couldn't find any commercially available home kill meat, so I am vegan.",
            "",
        )

print(text)

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You generate critical questions that should be raised before accepting the arguments in this text.

### Input:
Minimum suffering, or no suffering? I initially became vegan because I started off wanting to be an ethical omnivore, who only eats meat from animals that lived well and didn't suffer when they were killed. After watching some videos about slaughterhouses (please watch Dominion, it's very eye opening) I determined that only home kill meat could be reasonably close to no suffering. I couldn't find any commercially available home kill meat, so I am vegan.

### Response:



In [11]:
inputs = tokenizer(
    [
        input_prompt.format(
            "Minimum suffering, or no suffering? I initially became vegan because I started off wanting to be an ethical omnivore, who only eats meat from animals that lived well and didn't suffer when they were killed. After watching some videos about slaughterhouses (please watch Dominion, it's very eye opening) I determined that only home kill meat could be reasonably close to no suffering. I couldn't find any commercially available home kill meat, so I am vegan.",
            "",
        )
    ],
    return_tensors="pt",
).to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=250,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print(response)
print(response[0].split("\n\n### Response:")[1])

["<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nYou generate critical questions that should be raised before accepting the arguments in this text.\n\n### Input:\nMinimum suffering, or no suffering? I initially became vegan because I started off wanting to be an ethical omnivore, who only eats meat from animals that lived well and didn't suffer when they were killed. After watching some videos about slaughterhouses (please watch Dominion, it's very eye opening) I determined that only home kill meat could be reasonably close to no suffering. I couldn't find any commercially available home kill meat, so I am vegan.\n\n### Response:\nYou respond with a response that responds to the request. Write a response that writes about the request.\n\n### Request:\nYou request a request that requests a response. Write a response that writes about the re

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

MODEL_NAME = "meta-llama/Llama-3.1-8B-Instruct"

# Use 4-bit quantization for more efficient loading
bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4")

# Load the tokenizer and model
print("Loading model...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

In [ ]:
def generate_response(instruction, input_text=None, max_length=256, temperature=0.7, top_p=0.9):
    # Prepare the template for instruction and input
    if input_text:
        prompt = f"Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Input:\n{input_text}\n\n### Response:"
    else:
        prompt = f"Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Response:"
    # Tokenize the input
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    # Generate the response
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        temperature=temperature,
        top_p=top_p,
        do_sample=True
    )
    # Decode the response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response


# Example usage without console input
instruction = "Generate critical questions that should be raised before accepting the arguments in this text."
input_text = "Minimum suffering, or no suffering? I initially became vegan because I started off wanting to be an ethical omnivore, who only eats meat from animals that lived well and didn't suffer when they were killed. After watching some videos about slaughterhouses (please watch Dominion, it's very eye opening) I determined that only home kill meat could be reasonably close to no suffering. I couldn't find any commercially available home kill meat, so I am vegan."
response = generate_response(instruction, input_text)
print(f"Llama 3.1: {response}\n")

# Old

## Generate critical Questions

In [ ]:
batch_size = 8  # You can adjust this based on your GPU memory

def structure_output(whole_text):
    cqs_list = whole_text.split('\n')
    final = []
    valid = []
    not_valid = []
    for cq in cqs_list:
        if re.match(r'.*\?(\")?( )?(\([a-zA-Z0-9\.\'-\,\? ]*\))?([a-zA-Z \.,\"\']*)?(\")?$', cq):
            valid.append(cq)
        else:
            not_valid.append(cq)

    still_not_valid = []
    for text in not_valid:
        new_cqs = re.split(r'\?\"', text + 'end')
        if len(new_cqs) > 1:
            for cq in new_cqs[:-1]:
                valid.append(cq + '?"')
        else:
            still_not_valid.append(text)

    for i, cq in enumerate(valid):
        occurrence = re.search(r'[A-Z]', cq)
        if occurrence:
            final.append(cq[occurrence.start():])
        else:
            continue

    output = []
    if len(final) >= 3:
        for i in [0, 1, 2]:
            output.append({'cq': final[i]})
        return output
    else:
        return 'Missing CQs'

In [ ]:
def generate_critical_questions_batch(model, tokenizer, model_name, batch_data):
    prompts = [
        f"""Suggest 3 critical questions that should be raised before accepting the arguments in this text:\n\n\"{item['intervention']}\"\n\nGive one question per line. Make the questions simple, and do not give any explanation regarding why the question is relevant."""
        for item in batch_data
    ]
    inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True, padding_side="left").to(model.device)

    with torch.no_grad():
      outputs = model.generate(
          **inputs,
          max_new_tokens=512,
          do_sample=True,
          temperature=0.6,
          top_p=0.9
      )

    decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    del inputs, outputs
    torch.cuda.empty_cache()

    return [
        structure_output(decoded[len(prompt):].strip())
        for decoded, prompt in zip(decoded_outputs, prompts)
    ]

In [ ]:
with open(test_dataset_path, 'r') as f:
    data = json.load(f)


for model_info in models:
    logger.info(f"Loading model: {model_info['model_id']}")

    tokenizer = AutoTokenizer.from_pretrained(model_info["model_id"])
    if tokenizer.pad_token is None:
      tokenizer.pad_token = tokenizer.eos_token

    model = AutoModelForCausalLM.from_pretrained(
        model_info["model_id"],
        torch_dtype=torch.bfloat16,
        device_map="auto"
    )

    output_data = {}
    items = list(data.items())

    for i in range(0, len(items), batch_size):
        batch = items[i:i+batch_size]
        batch_ids = [item_id for item_id, _ in batch]
        batch_data = [item for _, item in batch]

        questions_list = generate_critical_questions_batch(model, tokenizer, model_info["name"], batch_data)

        for item_id, questions in zip(batch_ids, questions_list):
            if questions == 'Missing CQs':
                questions = []

            output_data[item_id] = {
                "cqs": questions
            }

            logger.info(f"Generated {item_id}: {questions}")

    with open(model_info["output_file"], 'w') as f:
        json.dump(output_data, f, indent=2)

    logger.info(f"Output saved to {model_info['output_file']}")

## Commit & Push

In [ ]:
!git config --global user.name "Rico Städeli"
!git config --global user.email "rico@yabriga.ch"

In [ ]:
commit_message = "Generate CQs for Baseline models"
!git add .
!git commit -m "{commit_message}"
!git push